In [1]:
import IPython
import reboting
from reboting import CouldntSaveDataException
from reboting import CsvDownloadAndParsingException
from reboting import CouldNotDownloadFileException
from reboting import CouldntCreateVisualException
from reboting import DataTooBigException
from reboting import NoVisualsPossible
import requests
import pandas as pd
import time

go over all queries for each bundesland

- for every item ask the bot with url
- and then analyze the db

In [4]:
stats = { 'total' : {
    'total' : 0,
    'success' : 0
}}
config = {
    'vienna' : '&l=http://sws.geonames.org/2761369/&l=http://sws.geonames.org/2761367/&l=http://sws.geonames.org/2761333/&l=http://sws.geonames.org/2761368/',
    'styria' : '&l=http://sws.geonames.org/2764581/',
    'lower_austria' : '&l=http://sws.geonames.org/2770542/',
    'upper_austria' : '&l=http://sws.geonames.org/2769848/',
    'tirol' : '&l=http://sws.geonames.org/2763587/&l=http://sws.geonames.org/2763586/&l=http://sws.geonames.org/2763588/',
    'vorarlberg' : '&l=http://sws.geonames.org/2762300/',
    'carinthia' : '&l=http://sws.geonames.org/2774686/',
    'burgenland' : '&l=http://sws.geonames.org/6955321/&l=http://sws.geonames.org/2781194/',
    'salzburg' : '&l=http://sws.geonames.org/2766824/&l=http://sws.geonames.org/2766818/&l=http://sws.geonames.org/2766823/&l=http://sws.geonames.org/7871495/'
}
for key, value in config.items():
    #print(key+" val: "+value)
    offset=0
    url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
    r = requests.get(url)
    resp = r.json()  
    stats[key]={ 'total': resp['total'], 'success' : 0}
    stats['total']['total'] = stats['total']['total']+resp['total']
    #lets start the for loop again
    for i in range(0,resp['total']):        
        offset=i        
        url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
        time.sleep(3)
        try: 
            r = requests.get(url)
            resp = r.json()  
            #hardcoded userid
            user_id='10213970072671093'
            searchresult = resp['results'][0]
            data_desc = {
                    "name" : "No name available",
                    "description" : "No description available",
                    "publisher" : "no publisher available",
                    "portal" : "No Portal available",
                    "url": searchresult['url'],
                    "user_id" : user_id
            }
            #print(str(i)+" "+searchresult['url'])
            if searchresult['dataset'] is not None:
                if searchresult['dataset']['dataset_name'] is not None:
                    data_desc['name'] = searchresult['dataset']['dataset_name']
                if searchresult['dataset']['dataset_description'] is not None:
                    data_desc['description'] = searchresult['dataset']['dataset_description']
            if searchresult['portal'] is not None:
                data_desc['portal'] = searchresult['portal']
            if searchresult['dataset']['publisher'] is not None:
                data_desc['publisher'] = searchresult['dataset']['publisher']
            try:
                slug = reboting.checkforknowncsv(data_desc = data_desc)
                url = 'https://doh.23degrees.io/viz/'+slug
                #print(url)
                stats[key]['success'] = stats[key]['success']+1
                stats['total']['success'] = stats['total']['success']+1            
            except CouldntSaveDataException as e:
                print("ERROR: couldnt save data on 23 degree server")
            except CsvDownloadAndParsingException as e:
                print("ERROR: something went wrong when downloading the file")
            except CouldNotDownloadFileException as e:
                print("ERROR: couldnt download file")
            except CouldntCreateVisualException as e:
                print("ERROR: couldnt create visual")
            except DataTooBigException as e:
                print("ERROR: too many lines: "+str(e))   
            except NoVisualsPossible as e:
                print("ERROR: no visual possible: "+str(e))
            print(str(i)+" success: "+str(stats['total']['success']))
        except Exception as e:
            print("new error: "+str(e))

a-map-for-registerzaehlung-2011-zaehlbezirke-1-wien-u-total-8
0 success: 1
a-map-for-migrationsstroeme-nach-migrationshintergrund-seit-2002-gemeindebezirke-wien-sex-7
1 success: 2
a-map-for-migrationsstroeme-nach-nationalitaet-seit-2002-gemeindebezirke-wien-sex-2
2 success: 3
a-map-for-bevoelkerungsprognose-2014-bis-2024-zaehlbezirke-1-wien-sex-1
3 success: 4
new error: Expecting value: line 1 column 1 (char 0)
a-map-for-bundespraesidentenwahlen-wien-t-4
5 success: 5
a-horizontal-barchart-for-migrationsstroeme-nach-migrationshintergrund-seit-2002-wien-sex-1
6 success: 6
a-map-for-nationalratswahlen-wien-t-4
7 success: 7
a-map-for-nationalratswahlen-wien-t-6
8 success: 8
a-horizontal-barchart-for-wiener-linien-routingservice-wien-haltestellen-id-1
9 success: 9
a-map-for-hunde-pro-bezirk-wien-postal-code-1
10 success: 10
a-map-for-bezirksvertretungswahlen-wien-t-4
11 success: 11
a-map-for-registerzaehlung-2011-zaehlbezirke-4-wien-whg-total-1
12 success: 12
a-map-for-registerzaehlung-2011

KeyboardInterrupt: 

In [ ]:
stats